<a href="https://colab.research.google.com/github/zYuanXiao/sketch-to-comics/blob/main/Dict/t2imodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U git+https://github.com/huggingface/diffusers.git
!pip install -U controlnet_aux==0.0.7 # for conditioning models and detectors
!pip install transformers accelerate safetensors
!pip install xformers



  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-avl86d7c
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-avl86d7c
  Resolved https://github.com/huggingface/diffusers.git to commit f00a995753732210a696de447cd0db80e181c30a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.34.0.dev0-py3-none-any.whl size=3602326 sha256=fefff0f4f1d5ab878552f8da340bd058dfe4602980f56c6bd771c87abaf5dde7
  Stored in directory: /tmp/pip-ephem-wheel-cache-h3q6ifox/wheels/d2/5c/5f/16639722ea17ecb73ab461b81718584bac08af2801619786b9
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.32.2
    Uninstalling diffusers-0.32.2:
      Successfully uninstalled diffusers-0.32.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.4/202.4 kB

In [2]:
from PIL import Image
from IPython.display import display
from google.colab import drive
import os
import cv2
import numpy as np
import torchvision.transforms as transforms

In [3]:
from diffusers import StableDiffusionXLAdapterPipeline, T2IAdapter, EulerAncestralDiscreteScheduler, AutoencoderKL
from diffusers.utils import load_image, make_image_grid
from controlnet_aux.canny import CannyDetector
import torch

# load adapter
adapter = T2IAdapter.from_pretrained("TencentARC/t2i-adapter-canny-sdxl-1.0", torch_dtype=torch.float16, varient="fp16").to("cuda")

# load euler_a scheduler
model_id = 'stabilityai/stable-diffusion-xl-base-1.0'
euler_a = EulerAncestralDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
vae=AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipe = StableDiffusionXLAdapterPipeline.from_pretrained(
    model_id, vae=vae, adapter=adapter, scheduler=euler_a, torch_dtype=torch.float16, variant="fp16",
).to("cuda")
pipe.enable_xformers_memory_efficient_attention()

canny_detector = CannyDetector()


/usr/local/lib/python3.11/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.11/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segmen

config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/316M [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/5.14G [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
image_path = '/content/drive/MyDrive/testpicture/test1.png'
img_rgb = cv2.imread(image_path)
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

blur = cv2.GaussianBlur(img_gray, (7, 7), 1.4)
edges = cv2.Canny(blur, threshold1=120, threshold2=240)

num, lbl, stats, _ = cv2.connectedComponentsWithStats(edges, connectivity=8)
min_size = 40       # 像素点个数 < 40 的边片段全部丢弃
mask = np.zeros_like(edges)
for i in range(1, num):           # 0 是背景
    if stats[i, cv2.CC_STAT_AREA] >= min_size:
        mask[lbl == i] = 255
edges_clean = mask
edges_bgr = cv2.cvtColor(edges_clean, cv2.COLOR_GRAY2BGR)
out_path = '/content/girl_edges_canny.jpg'
cv2.imwrite(out_path, edges_bgr)

True

In [25]:
image_path = '/content/girl_edges_canny.jpg'
image1 = Image.open(image_path)

prompt = "anime style, high quality, same pose, disappointed, manga illustration, smooth and simple line, vivid, white background"
negative_prompt = "graphic, text,  crayon, graphite, abstract, glitch, deformed, mutated, ugly, disfigured,extra digit, fewer digits, cropped, worst quality, low quality, glitch"
gen_images = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=image1,
    num_inference_steps=30,
    adapter_conditioning_scale=0.8,
    guidance_scale=7.5,
).images[0]
gen_images.save('/content/drive/MyDrive/testpicture/result1.png')

  0%|          | 0/30 [00:00<?, ?it/s]